# Quick Start

This notebook serves as a tutorial and illustrates how to use Path4GMNS step by step.

## 1. Install Path4GMNS

In [ ]:
pip install path4gmns

## 2. Prepare Input Data

node.csv and link.csv complying GMNS is the minimum requirement for you to get started. You can either download the sample data sets with the built-in function or prepare your own data set.

In [ ]:
import path4gmns as pg

download_sample_data_sets() will download five sample data sets including ASU, Braess Paradox, Chicago Sketch, Lima Network, Sioux Falls, and Two Corridors. Each of them contains the following files.

1. node.csv
2. link.csv
3. demand.csv
4. settings.yml (for multimodal analyses)
5. settings.csv (for DTALite)

In [ ]:
pg.download_sample_data_sets()

Navigate to the target data set directory. In this tutorial, we will use the [data set](https://github.com/jdlph/Path4GMNS/tree/dev/tests) where this Jupyter notebook is located for simplicity. You can check your present working directory (PWD) by the following commnad.

In [ ]:
pwd

If you are working in an online environment, e.g., Google Colab. You can move to one of the downloaded sample data sets, say ASU, via the following lines. 

In [ ]:
import os
os.chdir('data/Chicago_Sketch')

## 3. The Basic Functionalities
### 3.1 Get the Shortest Path between Two Nodes
Find the (static) shortest path according to travel time or distance and output it in the format of a sequence of node/link IDs.

In [ ]:
# create the network object by reading node.csv and link.csv
# no need to load demand file for equity evaluation
network = pg.read_network()

# node path from node 1 to node 2 measured by travel time
print('\nshortest path (node id) from node 1 to node 2, '
      +network.find_shortest_path(1, 2))
# link path from node 1 to node 2 measured by travel time
print('\nshortest path (link id) from node 1 to node 2, '
      +network.find_shortest_path(1, 2, seq_type='link'))

With v0.9.10, you can find the shortest path between any two different nodes in distance by specifying cost_type as 'distance'. The distance unit will be consistent with the one passed to read_network() as stated below.

In [ ]:
network = pg.read_network()

# node path from node 1 to node 2 measured by distance. distance is in 'mile'
print('\nshortest path (node id) from node 1 to node 2, '
      +network.find_shortest_path(1, 2, cost_type='distance'))
# link path from node 1 to node 2 measured by distance. distance is in 'mile'
print('\nshortest path (link id) from node 1 to node 2, '
      +network.find_shortest_path(1, 2, seq_type='link', cost_type='distance'))

Path4GMNS uses **mile** and **mile per hour (mph)** as the default units for length and speed in its internal calculation and all outputs. If your link.csv features different length and speed units, please specify them in read_network(). The supported units are list as blow.

 **Length Units**
 * meter and m
 * kilometer and km
 * mile and mi

 **Speed Units**
 * kmh or kph
 * mph
 
 You would have to do manual conversion if your units are not among the above ones.

In [ ]:
# the default are length_unit='mile' and speed_unit='mph'
network = pg.read_network(length_unit='meter', speed_unit='kph')

# node path from node 1 to node 2 measured by travel time
print('\nshortest path (node id) from node 1 to node 2, '
      +network.find_shortest_path(1, 2))
# link path from node 1 to node 2 measure by travel time
print('\nshortest path (link id) from node 1 to node 2, '
      +network.find_shortest_path(1, 2, seq_type='link'))

# node path from node 1 to node 2 measured by distance. distance is in 'meter'
print('\nshortest path (node id) from node 1 to node 2, '
      +network.find_shortest_path(1, 2, cost_type='distance'))
# link path from node 1 to node 2 measure by distance. distance is in 'meter'
print('\nshortest path (link id) from node 1 to node 2, '
      +network.find_shortest_path(1, 2, seq_type='link', cost_type='distance'))

### 3.2 Retrieve the Shortest Path Tree from a Source Node

Starting from v0.9.10, you can use get_shortest_path_tree() instead of repeatedly calling find_shortest_path() to get the shortest paths from a source node to any other nodes in the network. Its usage is very similar to the above get_shortest_path().

In [ ]:
network = pg.read_network()

# get shortest path tree (in node sequences) from node 1 (cost is measured by time)
sp_tree_node = network.get_shortest_path_tree(1)
# retrieve the shortest path from the source node (i.e., node 1) to node 2
print(f'shortest path (node id) from node 1 to node 2: {sp_tree_node[2]}')
# retrieve the shortest path from the source node (i.e., node 1) to node 3
print(f'shortest path (node id) from node 1 to node 3: {sp_tree_node[3]}')

# get shortest path tree (in link sequences) from node 1 (cost is measured by time)
sp_tree_link = network.get_shortest_path_tree(1, seq_type='link')
# retrieve the shortest path from the source node (i.e., node 1) to node 2
print(f'shortest path (link id) from node 1 to node 2: {sp_tree_link[2]}')
# retrieve the shortest path from the source node (i.e., node 1) to node 3
print(f'shortest path (link id) from node 1 to node 3: {sp_tree_link[3]}')

You can get the distance-based shortest path tree as well by passing cost_type='distance'.

In [ ]:
network = pg.read_network()

# get shortest path tree (in node sequences) from node 1 (cost is measured by distance)
sp_tree_node = network.get_shortest_path_tree(1, cost_type='distance')
# retrieve the shortest path from the source node (i.e., node 1) to node 2
print(f'shortest path (node id) from node 1 to node 2: {sp_tree_node[2]}')
# retrieve the shortest path from the source node (i.e., node 1) to node 3
print(f'shortest path (node id) from node 1 to node 3: {sp_tree_node[3]}')

# get shortest path tree (in link sequences) from node 1 (cost is measured by distance)
sp_tree_link = network.get_shortest_path_tree(1, seq_type='link', cost_type='distance')
# retrieve the shortest path from the source node (i.e., node 1) to node 2
print(f'shortest path (link id) from node 1 to node 2: {sp_tree_link[2]}')
# retrieve the shortest path from the source node (i.e., node 1) to node 3
print(f'shortest path (link id) from node 1 to node 3: {sp_tree_link[3]}')

### 3.3 Perform Path-Based UE Traffic Assignment

OD demand matrix and zone information is crucial in conducting traffic assignment are essential to perform this functionality. The demand file is presumed to be given and zone information is provided along node.csv. The latter is the current design of GMNS and is endorsed in [OSM2GMNS](https://github.com/jiawlu/OSM2GMNS).

The demand will be loaded by default via pg.read_demand(). If demand.csv is missing, pg.read_demand() will automatically conduct the following attempts.
1. try to load the synthetic demand (as syn_demand.csv and zones as syn_zone.csv).
2. synthesize demand (and zones) if the previous attempt fails.

Then you can conduct traffic assignment via the following lines.

In [ ]:
network = pg.read_network()
pg.read_demand(network)

# specify the parameters for traffic assignment
column_gen_num = 10
column_upd_num = 10

# path-based UE only
pg.find_ue(network, column_gen_num, column_upd_num)

# if you do not want to include geometry info in the output file,
# use pg.output_columns(network, False)
# output column information to route_assignment.csv
pg.output_columns(network)
# output link performance to link_performance.csv
pg.output_link_performance(network)

The demand file is specified in settings.yml as demand.csv (see 4.1 Sample setting.yml for details). You can change it to any demand file in your PWD. This design actually allows us to load multiple demand files simultaneously corresponding to different period and agent_type. We will elaborate it in a future release.

The code snippet below demonstrates how to adaptively the UE convergency directly through rel_gap_tolerance. find_ue() will terminate when either column_upd_num or rel_gap_tolerance is reached.

In [ ]:
network = pg.read_network()
pg.read_demand(network)

# specify the parameters for traffic assignment
column_gen_num = 20
column_upd_num = 20

# the default value of rel_gap_tolerance is 0.0001 if not specified
rel_gap = pg.find_ue(network, column_gen_num, column_upd_num, rel_gap_tolerance = 0.001)
print(f'the final relative UE gap is {rel_gap:.4%}')

# if you do not want to include geometry info in the output file,
# use pg.output_columns(network, False)
pg.output_columns(network)
pg.output_link_performance(network)

### 3.4 Conduct Dynamic Traffic Simulation

Traffic simulation requires routing decision for each agent, which comes from the UE traffic assignment.

In [ ]:
network = pg.read_network()
pg.read_demand(network)

# UE + DTA
column_gen_num = 10
column_upd_num = 10
pg.find_ue(network, column_gen_num, column_upd_num)
pg.perform_simple_simulation(network)
print('complete dynamic simulation.\n')

print('writing agent trajectories')
pg.output_agent_trajectory(network)

If you have route_assignment.csv (i.e., columns) from a previous run or DTALite, you can bypass perform_column_generation() and directly load it to conduct simulation.

In [ ]:
# no need to load demand file as we will infer the demand from columns
network = pg.read_network()

# load existing UE result
pg.load_columns(network)

# DTA
pg.perform_simple_simulation(network)
print('complete dynamic simulation.\n')

print('writing agent trajectories')
pg.output_agent_trajectory(network)

If you are still interested in traffic simulation using shortest paths, it can be achieved by setting column_gen_num as 1 and column_upd_num as 0 illustrated below.

In [ ]:
network = pg.read_network()
pg.read_demand(network)

# the following setting will set up the shortest path for each agent
column_gen_num = 1
column_upd_num = 0
pg.find_ue(network, column_gen_num, column_upd_num)
pg.perform_simple_simulation(network)
print('complete dynamic simulation.\n')

print('writing agent trajectories')
pg.output_agent_trajectory(network)

### 3.5 Run ODME

You can utilize traffic observations to calibrate UE results. Traffic observations in terms of traffic count at link, or zone origin, or zone destination must be specified in measurement.csv.

In [ ]:
network = pg.read_network()

pg.read_demand(network)

# path-based UE
column_gen_num = 20
column_upd_num = 20
pg.find_ue(network, column_gen_num, column_upd_num)

# ODME
pg.read_measurements(network)
pg.conduct_odme(network, 20)

# output column information to route_assignment.csv
pg.output_columns(network)
# output link performance to link_performance.csv
pg.output_link_performance(network)

Similar to DTA, the UE results as input to ODME could come from existing runs. Code snippet will be skipped here. Please refer to the previous section for details.

### 3.6 Evaluate Accessibility

Accessiblity defines where you can go given a time budget and a transportation mode (e.g., auto). You can find the number of accessible zones from each zone (zone_accessibility.csv) along with the free flow travel time for each OD pair **in minutes** (od_accessibility.csv).

The default mode is 'auto' and the default time budget is 240 minutes. We will come back to multimodal evaluation in a later section. Zone information is necessary for accessibility evaluation.

In [ ]:
# no need to load demand file for accessibility evaluation
network = pg.read_network()

pg.evaluate_accessibility(network, single_mode=True)

### 3.7 Evaluate Equity

Transportation equity is accessibility with respect to different demographics. Path4GMNS rovides the following simple info and statistics on equity given a time budget and a segmentation of zones.

1. accessible zones.
2. min accessibility. This metric refers to the zone with the minimum number of accessible zones. 
3. max accessibility. This metric refers to the zone with the maximum number of accessible zones. 
4. mean accessibility. The average number of accessible zones over a bin of zones (corresponding to a specific demographic) given a time budget and a transportation mode.

Similar to accessiblity evaluation, the default mode is 'auto' but the default time budget is 60 minutes. Zone information is still required here.

In [ ]:
# no need to load demand file for equity evaluation
network = pg.read_network()

pg.evaluate_equity(network, single_mode=True)

## 4. Move Foward to Multimodal Evaluation

In order to perform multimodal evaluation, the corresponding modes (i.e., agent types) must be presented in settings.yml. It will be parsed by pyyaml (5.1 or higher) to the Python engine at run-time. 

pyyaml must be installed before running any multimodal evaluation.

In [ ]:
pip install pyyaml

### 4.1 Sample setting.yml
A sample file looks like blow. You can start from here and modify it towards your own needs. **Note that** the default agent type which is **'a' as type and 'auto' as name must be present in this file**. Otherwise, you will encounter an exception with message "No AGENT type: a". We will relax this requirement in a later release.

```yaml
agents:
  - type: a
    name: auto
    vot: 10
    flow_type: 0
    pce: 1
    free_speed: 60
    use_link_ffs: true
  - type: w
    name: walk
    vot: 10
    flow_type: 0
    pce: 1
    free_speed: 10
    use_link_ffs: false

demand_periods:
  - period: AM
    time_period: 0700-0800

demand_files:
  - file_name: demand.csv
    period: AM
    agent_type: a
```

You can download a sample settings.yml (to your PWD) if you do not have one by the following command.

In [ ]:
pg.download_sample_setting_file()

### 4.2 Get the Shortest Path between Two Nodes under a Specific Mode
In 3.1, we used this functionlity implicitly for the default mode, which is 'a' or equivalently 'auto'. Now with 'walk' defined in settings.yml, we are able to find the shortest path under mode 'w' or 'walk'.

In [ ]:
network = pg.read_network()

# get the shortest path measured by travel time
print('\nshortest path (node id) from node 1 to node 2, '
      +network.find_shortest_path(1, 2, mode='w'))
print('\nshortest path (link id) from node 1 to node 2, '
      +network.find_shortest_path(1, 2, mode='w', seq_type='link'))

# get the shortest path measured by distance
print('\nshortest path (node id) from node 1 to node 2, '
      +network.find_shortest_path(1, 2, mode='w', cost_type='distance'))
print('\nshortest path (link id) from node 1 to node 2, '
      +network.find_shortest_path(1, 2, mode='w', seq_type='link', cost_type='distance'))

For this test data set, 3.1 and 4.2 have the same results as each link is open to all modes, i.e., their "allowed_uses" are "all". 

### 4.3 Obtain the Mode Specific Shortest Path Tree

Similar to get_shortest_path(), the shortest path tree can be mode specific as well.

In [ ]:
network = pg.read_network()

# get shortest path tree (in node sequences) from node 1 (cost is measured by time) under mode 'w'
sp_tree_node = network.get_shortest_path_tree(1, mode='w')
# retrieve the shortest path from the source node (i.e., node 1) to node 2
print(f'shortest path (node id) from node 1 to node 2: {sp_tree_node[2]}')
# retrieve the shortest path from the source node (i.e., node 1) to node 3
print(f'shortest path (node id) from node 1 to node 3: {sp_tree_node[3]}')

# get shortest path tree (in link sequences) from node 1 (cost is measured by distance) under mode 'w'
sp_tree_link = network.get_shortest_path_tree(1, mode='w', seq_type='link', cost_type='distance')
# retrieve the shortest path from the source node (i.e., node 1) to node 2
print(f'shortest path (link id) from node 1 to node 2: {sp_tree_link[2]}')
# retrieve the shortest path from the source node (i.e., node 1) to node 3
print(f'shortest path (link id) from node 1 to node 3: {sp_tree_link[3]}')

### 4.4 Perform Multimodal Accessibility Evaluation

It is very similar to what we have done in 3.6 for a single-mode analysis.

In [ ]:
# no need to load demand file for accessibility evaluation
network = pg.read_network()

pg.evaluate_accessibility(network)

### 4.5 Perform Multimodal Equity Evaluation

By removing "single_mode=True" from the arugment list in evaluate_equity(), multimodal evaluation will be triggered for all modes defined in settings.yml

In [ ]:
# no need to load demand file for equity evaluation
network = pg.read_network()

pg.evaluate_equity(network)

## 5. Advanced Features

### 5.1 In Case of Special Events

A special event often comes with capacity reduction over affected links. You can introduce one special event for each demand period in settings.yml as below. 

```yaml
demand_periods:
  - period: AM
    time_period: 0700-0800
    special_event:
      name: work_zone
      enable: true
      affected_links:
        - link_id: 1
          capacity_ratio: 0.5
        - link_id: 2
          capacity_ratio: 0.4
        - link_id: 3
          capacity_ratio: 0.6
        - link_id: 4
          capacity_ratio: 0
```

If the original capacity of an affected link i is C, its capacity then will be r * C with a reduction ratio of r when a special event is present. For an affected link, setting its capacity_ratio to 0 is equivalent to removing it from the entire demand period. You can turn on or off a special event by setting enable to true or false.

Then you can conduct traffic assignment (as shown in 3.3) and see the impact from a special event.

### 5.2 Accessibility Considering Time-Dependent Link Travel Time

In 3.6 and 4.4, accessiblity is evluated using the link free flow travel time, which is determined by link length and link free-flow speed under a specific mode. 

Link travel time varies over time so does accessibility. When the time-dependent accessibility is of interested, time-dependent link travel time (i.e., VDF_fftt from a given demand period in link.csv) will come into play by overwriting the (static) link free-flow speed (from either link.csv or settings.yml. both are denoted as "free_speed").

In [ ]:
network = pg.read_network()

# time-dependent accessibility under the default mode auto (i.e., 'a')
# for demand period 0 (i.e., VDF_fftt1 in link.csv will be used in the evaluation)
pg.evaluate_accessibility(network, single_mode=True, time_dependent=True)

# if you would like to evaluate accessibility under a target mode, say walk, then
pg.evaluate_accessibility(network, single_mode=True, mode='w', time_dependent=True)

## 6. DTALite

Path4GMNS also serves as an API to the C++-based DTALite to conduct various multimodal traffic assignments including,

    0: Link-based UE,
    1: Path-based UE,
    2: UE + Dynamic Traffic Assignment (DTA),
    3: OD Matrix Estimation (ODME).

Here we only demonstrate how to perform path-based UE (i.e., mode 1) using DTALite from Path4GMNS. You would need [settings.csv](https://github.com/jdlph/Path4GMNS/blob/dev/tests/settings.csv) to get started.

In order to use DTALite embedded in Path4GMNS, run-time support OpenMP must be installed in the first space. See [Installation](https://path4gmns.readthedocs.io/en/latest/installation.html) for details.

In [ ]:
mode = 1
column_gen_num = 10
column_upd_num = 10

pg.perform_network_assignment_DTALite(mode, column_gen_num, column_upd_num)

Please consider running the script calling this API using system terminal rather than the Python console for proper logging.

Starting from v0.9.6, you could conduct the above tasks using the [Multimodal DTALite](https://github.com/asu-trans-ai-lab/DTALite/tree/feature/multimodal). No parameters are needed. 

**Note that** running it with the current input will lead to *kernel crash* as it requires a **different [settings.yml](https://github.com/asu-trans-ai-lab/DTALite/blob/feature/multimodal/data/01_4_node_network/minimum_input/settings.yml)**! You can test this API with [this data set](https://github.com/asu-trans-ai-lab/DTALite/tree/feature/multimodal/data). 

In [ ]:
pg.run_DTALite()